<h1>Example: Convolutional Neural Network using TensorFlow (2.3.0) </h1>

Using Raw version - weights and biases

<h3>Convolution Neural Network</h3>

![CNN](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/CNN.png)

<h3>MNIST Dataset</h3>

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. 

![MNIST](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/MnistExamples.png)

<h4>Step 1: Import Libraries</h4>

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import pandas as pd

print(tf.__version__)

2.3.0


<h4>Step 2: Parameters</h4>

In [2]:
# MNSIT dataset parameters (No need of num_input and dropout as in version 1.15)
num_classes=10

# Training parameters

learning_rate=0.001
num_steps=500
batch_size=128
display_step=10

# Network parameters (No tf graph input through placeholder as in version 1.15)

conv1_filters=32 # first layer
conv2_filters=64 # second layer
fc1_units=1024 #neurons in fully connected layer

<h4> Step3: MNIST dataset from Keras dataset </h4>

In version 1.15, we used tensorflow.examples.tutorials.mnist packaage, which is no more available in version 2, so it is better to use datasets from keras 

In [3]:
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test)= mnist.load_data() # get training and testing data

#x is the data for images and y is the classes and labels

x_train,x_test = np.array(x_train,np.float32),np.array(x_test,np.float32) #First convert the data to float32
x_train,x_test = x_train/255,x_test/255 # Normalize the image value from [0,255] to [0,1]

In [4]:
# Shuffle and Batch Data
# -- In version 1.15, we used batch size in sess.run nad used mnist.train.next_batch(batch_size) but in 2.3.0 we will
# use tf.data for training data

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

<h4> Step4: Creating wrappers for conv2d and maxpool2d </h4>

Basically done for having control over weights and biases 

In [5]:
def conv2d(x,W,b,strides=1):
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME') #for weights
    x=tf.nn.bias_add(x,b) #adding biases
    return tf.nn.relu(x) #return with ReLu activation

def maxpool2d(x,k=2):
    mpool2d=tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
    return mpool2d

<h4> Step5: Defining and storing layers for weights and biases </h4>

In [6]:
# First we need random value generator to intialize weights

random_normal=tf.initializers.RandomNormal()

weights={
    'wc1':tf.Variable(random_normal([5,5,1,conv1_filters])), # Layer 1: 5x5 conv, 1 input, 32 filters
    'wc2':tf.Variable(random_normal([5,5,conv1_filters,conv2_filters])), # Layer 2: 5x5 conv, 32 inputs and 64 filters
    'wd1':tf.Variable(random_normal([7*7*64,fc1_units])), #fully connected layer (7*7*64) input and 1024 neurons
    'out':tf.Variable(random_normal([fc1_units,num_classes]))
}

biases={
    'bc1':tf.Variable(tf.zeros([conv1_filters])),
    'bc2':tf.Variable(tf.zeros([conv2_filters])),
    'bd1':tf.Variable(tf.zeros([fc1_units])),
    'out':tf.Variable(tf.zeros([num_classes]))
}

<h4> Step6: Create Model and Define Loss and Accuracy functions </h4>

In [7]:
def conv_total(x):
    
    x=tf.reshape(x,[-1,28,28,1]) # Input =[-1,28,28,1] = 28x28x1 grayscale images
    
    conv1=conv2d(x,weights['wc1'],biases['bc1'])                             #output = [-1,28,28,32]
    conv1=maxpool2d(conv1,k=2)                                               #output = [-1,14,14,32]
    
    conv2=conv2d(conv1,weights['wc2'],biases['bc2'])                         #output = [-1,14,14,64]
    conv2=maxpool2d(conv2,k=2)                                               #output = [-1,7,7,64]
    
    fc1=tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])       #output = [-1,7*7*64]
    
    fc1=tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])                  #output = [-1,1024]
    fc1=tf.nn.relu(fc1) #apply Relu for non-linearity
    
    out=tf.add(tf.matmul(fc1,weights['out']),biases['out'])                  #output = [-1,10]
    
    return tf.nn.softmax(out) #softmax to normalize logits

In [8]:
def cross_entropy_loss(y_pred,y_true):
    y_true=tf.one_hot(y_true,depth=num_classes) #encode labels by one hot ve
    y_pred=tf.clip_by_value(y_pred,1e-9,1.) #use clip prediction to avoid log(0) error
    
    return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))

def accuracy(y_pred,y_true):
    correct_prediction=tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),axis=-1)

optimizer=tf.optimizers.Adam(learning_rate)

<h4> Step7: Optimization Process </h4>

In [9]:
def optimization(x,y):
    with tf.GradientTape() as g: # computing optimization process as session run
        pred=conv_total(x) #using prediction funtion in conv_total()
        loss=cross_entropy_loss(pred,y) #compute loss
        
    train_var=list(weights.values()) + list(biases.values()) # variables update
    grad=g.gradient(loss,train_var) #compute gradients
    
    optimizer.apply_gradients(zip(grad,train_var)) #update weights and biases following gradients

<h4> Step8: Train Model </h4>

In [10]:
for step, (batch_x,batch_y) in enumerate(train_data.take(num_steps),1):
    optimization(batch_x,batch_y)
    
    if step % display_step==0:
        pred=conv_total(batch_x)
        loss=cross_entropy_loss(pred,batch_y)
        acc=accuracy(pred,batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

# Test model on validation set (accuracy)
pred = conv_total(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

step: 10, loss: 74.003220, accuracy: 0.835938
step: 20, loss: 41.783424, accuracy: 0.867188
step: 30, loss: 42.944805, accuracy: 0.921875
step: 40, loss: 25.742123, accuracy: 0.937500
step: 50, loss: 28.602489, accuracy: 0.914062
step: 60, loss: 10.795020, accuracy: 0.984375
step: 70, loss: 21.607037, accuracy: 0.968750
step: 80, loss: 26.993595, accuracy: 0.937500
step: 90, loss: 13.408720, accuracy: 0.968750
step: 100, loss: 8.451270, accuracy: 0.984375
step: 110, loss: 11.804845, accuracy: 0.976562
step: 120, loss: 14.731485, accuracy: 0.953125
step: 130, loss: 8.882578, accuracy: 0.976562
step: 140, loss: 17.711435, accuracy: 0.953125
step: 150, loss: 6.498240, accuracy: 0.984375
step: 160, loss: 2.394658, accuracy: 1.000000
step: 170, loss: 12.819140, accuracy: 0.960938
step: 180, loss: 5.203889, accuracy: 0.992188
step: 190, loss: 5.837789, accuracy: 0.984375
step: 200, loss: 11.276522, accuracy: 0.960938
step: 210, loss: 10.039032, accuracy: 0.984375
step: 220, loss: 3.018801, a